# Preprocesamiento

Se importan las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

Se carga el csv en un dataframe

In [2]:
data = pd.read_csv('../bd/data2.csv', dtype={'rut': str})
data

,rut,sexo,rangoEdad,plan,centroFav,tipoFav,visitas2018,visitas2019,visitas2020,visitas2021,visitas2022
0,25274441,Femenino,18 AÑOS O MENOS,COLMENA,CMD San Bernardo,Dental,0,0,0,1,0
1,25274447,Femenino,31 AÑOS - 40 AÑOS,CONSALUD,CM Parque Arauco,Médica,0,1,0,0,0
2,25274449,Femenino,31 AÑOS - 40 AÑOS,FONASA,Clínica Rancagua,Médica,0,1,0,0,0
3,25274450,Femenino,18 AÑOS O MENOS,CONSALUD,Clínica Elqui,Urgencia,0,0,0,0,0
4,25274458,Femenino,18 AÑOS O MENOS,BANMEDICA,Clínica Santiago,Urgencia,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3187257,18453648,Femenino,19 AÑOS - 30 AÑOS,FONASA,Telemedicina,Telemedicina,0,0,0,0,0
3187258,18453651,Femenino,19 AÑOS - 30 AÑOS,FONASA,Clínica Rancagua,Imagenologia,0,0,0,1,0
3187259,18453652,Masculino,19 AÑOS - 30 AÑOS,FONASA,CMD San Miguel - Gran Avenida,Médica,0,0,0,0,1
3187260,1845366,Masculino,19 AÑOS - 30 AÑOS,FONASA,CMD La Florida,Médica,0,0,0,0,1


Se crean los codificadores de los datos categóricos

In [3]:
le_rangoEdad = LabelEncoder()
le_sexo = LabelEncoder()
le_plan = LabelEncoder()
le_centro = LabelEncoder()
le_tipo = LabelEncoder()

Se codifican las variables

In [4]:
rangoEdad_encoded = le_rangoEdad.fit_transform(data['rangoEdad'])
sexo_encoded = le_sexo.fit_transform(data['sexo'])
plan_encoded = le_plan.fit_transform(data['plan'])
centro_encoded = le_centro.fit_transform(data['centroFav'])
tipo_encoded = le_tipo.fit_transform(data['tipoFav'])

encoded_df = data[['visitas2018','visitas2019','visitas2020','visitas2021','visitas2022']]
encoded_df.insert(0, column='rut', value=data[['rut']])
encoded_df.insert(1, column='sexo_encoded', value=sexo_encoded)
encoded_df.insert(2, column='rangoEdad_encoded', value=rangoEdad_encoded)
encoded_df.insert(3, column='plan_encoded', value=plan_encoded)
encoded_df.insert(4, column='centro_encoded', value=centro_encoded)
encoded_df.insert(5, column='tipo_encoded', value=tipo_encoded)

encoded_df

,rut,sexo_encoded,rangoEdad_encoded,plan_encoded,centro_encoded,tipo_encoded,visitas2018,visitas2019,visitas2020,visitas2021,visitas2022
0,25274441,0,0,1,44,0,0,0,0,1,0
1,25274447,0,2,2,26,7,0,1,0,0,0
2,25274449,0,2,4,54,7,0,1,0,0,0
3,25274450,0,0,2,50,12,0,0,0,0,0
4,25274458,0,0,0,55,12,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3187257,18453648,0,1,4,59,11,0,0,0,0,0
3187258,18453651,0,1,4,54,4,0,0,0,1,0
3187259,18453652,1,1,4,45,7,0,0,0,0,1
3187260,1845366,1,1,4,34,7,0,0,0,0,1


Se separan los datos en un set de entrenamiento (80%) y un set de pruebas (20%)

In [5]:
features = encoded_df.drop(['visitas2022'], axis=1)
labels = encoded_df[['visitas2022']]
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

Se elimina el rut de los sets de entrenamiento y prueba ya que no serán utilizados en el modelo

In [6]:
X_train.drop(['rut'], axis=1,inplace=True)
X_test.drop(['rut'], axis=1,inplace=True)

# Random Forest

Se crea el modelo de RandomForest

In [7]:
model = RandomForestClassifier()
model.fit(X_train,y_train.values.ravel())

RandomForestClassifier()

Se crean las predicciones

In [8]:
y_pred = model.predict(X_test)

Se muestran los indicadores del modelo

In [9]:
MSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
R2 = r2_score(y_test,y_pred)
Score = model.score(X_test,y_test)
print('MSE: ',MSE)
print('Score: ',Score)
print('R2_Score: ',R2)

MSE:  0.016330533617948475
Score:  0.999738019901075
R2_Score:  0.9987290492210135


# Postprocesamiento

Se agrega la predicción a un dataframe para facilitar su visualización

In [10]:
pred_data = pd.DataFrame(y_pred, columns=['pred_visitas2022'])
pred_data

,pred_visitas2022
0,0
1,1
2,0
3,0
4,0
...,...
637448,0
637449,0
637450,0
637451,1


Se unen los datos para entender de mejor manera los datos reales vs la predicción

In [11]:
pred_data = pd.DataFrame(y_pred, columns=['pred_visitas2022'])
final_data = X_test

final_data = final_data.assign(real_visitas2022=y_test)
final_data.reset_index(drop=True,inplace=True)

inverse_sexo = le_sexo.inverse_transform(final_data['sexo_encoded'])
final_data.insert(0, column='sexo', value=inverse_sexo)
final_data.drop('sexo_encoded',axis=1,inplace=True)

inverse_rangoEdad = le_rangoEdad.inverse_transform(final_data['rangoEdad_encoded'])
final_data.insert(1, column='rangoEdad', value=inverse_rangoEdad)
final_data.drop('rangoEdad_encoded',axis=1,inplace=True)

inverse_plan = le_plan.inverse_transform(final_data['plan_encoded'])
final_data.insert(2, column='plan', value=inverse_plan)
final_data.drop('plan_encoded',axis=1,inplace=True)

inverse_centro = le_centro.inverse_transform(final_data['centro_encoded'])
final_data.insert(3, column='centro', value=inverse_centro)
final_data.drop('centro_encoded',axis=1,inplace=True)

inverse_tipo = le_tipo.inverse_transform(final_data['tipo_encoded'])
final_data.insert(4, column='tipo', value=inverse_tipo)
final_data.drop('tipo_encoded',axis=1,inplace=True) 

final_df = pd.concat([final_data, pred_data],axis=1)
final_df.dropna(inplace=True)
final_df.reset_index()
final_df[['visitas2018','visitas2019','visitas2020','visitas2021','real_visitas2022','pred_visitas2022']] = final_df[['visitas2018','visitas2019','visitas2020','visitas2021','real_visitas2022','pred_visitas2022']].astype(int)
final_df

,sexo,rangoEdad,plan,centro,tipo,visitas2018,visitas2019,visitas2020,visitas2021,real_visitas2022,pred_visitas2022
0,Femenino,19 AÑOS - 30 AÑOS,FONASA,Clínica Santiago,Laboratorio,0,0,0,0,0,0
1,Masculino,19 AÑOS - 30 AÑOS,FONASA,Clínica Valparaíso,Médica,0,0,0,0,1,1
2,Femenino,51 AÑOS - 60 AÑOS,FONASA,CMD Puente Alto,Médica,1,0,0,0,0,0
3,Masculino,41 AÑOS - 50 AÑOS,NUEVA MAS VIDA,Clínica Temuco,Imagenologia,1,0,0,0,0,0
4,Femenino,19 AÑOS - 30 AÑOS,FONASA,Clínica Providencia,Imagenologia,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
637448,Masculino,41 AÑOS - 50 AÑOS,CONSALUD,Clínica Santiago,Hospitalización,0,0,0,0,0,0
637449,Femenino,51 AÑOS - 60 AÑOS,PARTICULAR,CMD Maipu,Médica,0,0,0,1,0,0
637450,Masculino,18 AÑOS O MENOS,FONASA,Clínica Rancagua,Laboratorio,0,0,0,0,0,0
637451,Femenino,19 AÑOS - 30 AÑOS,FONASA,CMD Temuco,Dental,0,0,0,0,1,1


Se muestra la suma y diferencia entre visitas reales del 2022 vs predecidas

In [12]:
real = int(final_df['real_visitas2022'].sum())
pred = int(final_df['pred_visitas2022'].sum())
total = real + pred
diff = real - pred

aciertos = pd.DataFrame()
aciertos['aciertos'] = np.where((final_df['real_visitas2022'] == final_df['pred_visitas2022']),1,0)

print('Suma de las visitas reales del 2022: ', real)
print('Suma de las visitas predichas del 2022: ', pred)
print('Diferencia entre visitas reales vs predichas: ', diff)
print('Total de las visitas 2022: ', total)
#Nota: Que el total de visitas sea menor al total de filas de la base de datos está bien, ya que no está sumando los 0s.
print('-------------')
print('Aciertos de la predicción: ', aciertos['aciertos'].sum())
print('Fallos de la predicción: ', (aciertos['aciertos']==0).sum())
print('Porcentaje de aciertos: ', ((aciertos['aciertos'].sum())*100)/len(final_df))
print('Porcentaje de fallos: ', (((aciertos['aciertos']==0).sum())*100)/len(final_df))

Suma de las visitas reales del 2022:  190153
Suma de las visitas predichas del 2022:  189995
Diferencia entre visitas reales vs predichas:  158
Total de las visitas 2022:  380148
-------------
Aciertos de la predicción:  637286
Fallos de la predicción:  167
Porcentaje de aciertos:  99.9738019901075
Porcentaje de fallos:  0.026198009892494035


Se crea una visualización de diferencia entre las visitas reales del 2022 vs las predecidas

In [13]:
final_df['diff_visitas2022'] = final_df['real_visitas2022'] - final_df['pred_visitas2022']
dif = final_df[final_df.diff_visitas2022 >= 1]
dif.head(50)

,sexo,rangoEdad,plan,centro,tipo,visitas2018,visitas2019,visitas2020,visitas2021,real_visitas2022,pred_visitas2022,diff_visitas2022
6033,Femenino,19 AÑOS - 30 AÑOS,CONSALUD,CD Talca,Dental,0,0,0,0,2,1,1
8694,Masculino,41 AÑOS - 50 AÑOS,FONASA,CM Parque Arauco,Dental,0,0,0,0,2,1,1
10777,Masculino,31 AÑOS - 40 AÑOS,CRUZ BLANCA,Clínica Rancagua,Procedimientos,0,0,0,0,2,1,1
15021,Masculino,41 AÑOS - 50 AÑOS,FONASA,Clínica Rancagua,Imagenologia,0,0,0,0,2,1,1
16092,Masculino,61 AÑOS - 74 AÑOS,BANMEDICA,CMD Conchali,Médica,0,0,0,0,2,1,1
22035,Femenino,31 AÑOS - 40 AÑOS,FONASA,CMD Pedro de Valdivia - Providencia,Imagenologia,0,0,0,0,2,1,1
24044,Femenino,31 AÑOS - 40 AÑOS,FONASA,Clínica Santiago,Médica,0,0,0,0,2,1,1
26735,Masculino,41 AÑOS - 50 AÑOS,FONASA,Clínica Iquique,Imagenologia,0,0,0,0,2,1,1
29819,Masculino,51 AÑOS - 60 AÑOS,CONSALUD,CD Huechuraba,Dental,0,0,0,0,2,1,1
40533,Masculino,19 AÑOS - 30 AÑOS,FONASA,Clínica Valparaíso,Imagenologia,0,0,0,0,2,1,1
